In [ ]:
import os
import glob
import h5py
import random
import numpy as np
from tqdm import tqdm
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve
from pathlib import Path
from astropy.visualization import ZScaleInterval
from PIL import Image


In [20]:
new_path = r'C:\Users\liaok\OneDrive\Desktop\毕设\data'

# 检查路径是否存在
if os.path.exists(new_path):
    os.chdir(new_path)
    print(f"工作目录已切换到：{new_path}")

工作目录已切换到：C:\Users\liaok\OneDrive\Desktop\毕设\data


### 导入数据（ori.png）

In [17]:
# 创建输出目录（修改为data_ori_png）
output_dir = 'data_ori_png'
os.makedirs(output_dir, exist_ok=True)

# 打开HDF5文件（网页6、网页7的HDF5处理方案）
with h5py.File('Galaxy10_DECals.h5', 'r') as file:
    images = file['images']

    # 随机选择2500个索引
    random_indices = random.sample(range(len(images)), 2500)

    # 处理流程优化（网页2、网页4的最佳实践）
    for idx in tqdm(random_indices, desc='处理进度', unit='张', colour='green'):
        image_data = images[idx]

        # 维度调整（原始代码逻辑保留）
        if image_data.shape == (256, 3, 256):
            image_data = np.transpose(image_data, (0, 2, 1))  # (256,256,3)

        # 转换为PIL图像对象（网页1、网页4推荐方案）
        pil_img = Image.fromarray(image_data.astype(np.uint8))

        # 高质量缩放（网页4建议使用LANCZOS插值）
        resized_img = pil_img.resize((512, 512), resample=Image.Resampling.LANCZOS)

        # 保存为PNG（网页2、网页4的标准保存方法）
        resized_img.save(os.path.join(output_dir, f'galaxy_image_{idx}_512.png'))

处理进度: 100%|██████████| 2500/2500 [23:01<00:00,  1.81张/s]


### ori.png->ori.fit

In [22]:
src_dir = "data_ori_png"
dst_dir = "data_ori_fit"
os.makedirs(dst_dir, exist_ok=True)

for filename in os.listdir(src_dir):
    if filename.endswith(".png") and "_512" in filename:
        new_name = filename.replace("_512", "")[:-4] + ".fit"
        src_path = os.path.join(src_dir, filename)
        dst_path = os.path.join(dst_dir, new_name)

        try:
            # 关键处理步骤
            with Image.open(src_path) as img:
                # 转换为单通道灰度图（确保二维）
                gray_img = img.convert('L')
                img_data = np.array(gray_img)

            # 验证数据维度
            if len(img_data.shape) != 2:
                raise ValueError(f"数据维度异常：{img_data.shape}，应确保输出二维数组")

            # 创建FITS结构
            hdu = fits.PrimaryHDU(img_data)
            hdu.writeto(dst_path, overwrite=True)
            print(f"成功转换: {filename} -> {new_name} (维度：{img_data.shape})")

        except Exception as e:
            print(f"转换失败 {filename}: {str(e)}")

成功转换: galaxy_image_10003_512.png -> galaxy_image_10003.fit (维度：(512, 512))
成功转换: galaxy_image_1000_512.png -> galaxy_image_1000.fit (维度：(512, 512))
成功转换: galaxy_image_10031_512.png -> galaxy_image_10031.fit (维度：(512, 512))
成功转换: galaxy_image_10034_512.png -> galaxy_image_10034.fit (维度：(512, 512))
成功转换: galaxy_image_10046_512.png -> galaxy_image_10046.fit (维度：(512, 512))
成功转换: galaxy_image_10061_512.png -> galaxy_image_10061.fit (维度：(512, 512))
成功转换: galaxy_image_10073_512.png -> galaxy_image_10073.fit (维度：(512, 512))
成功转换: galaxy_image_10081_512.png -> galaxy_image_10081.fit (维度：(512, 512))
成功转换: galaxy_image_10085_512.png -> galaxy_image_10085.fit (维度：(512, 512))
成功转换: galaxy_image_10090_512.png -> galaxy_image_10090.fit (维度：(512, 512))
成功转换: galaxy_image_10092_512.png -> galaxy_image_10092.fit (维度：(512, 512))
成功转换: galaxy_image_10093_512.png -> galaxy_image_10093.fit (维度：(512, 512))
成功转换: galaxy_image_10094_512.png -> galaxy_image_10094.fit (维度：(512, 512))
成功转换: galaxy_image_10101_51

### ori.fit->moxing.png

In [24]:
def fits_to_png_512(input_dir, output_dir, target_size=(512, 512)):
    """
    将指定目录下的FITS文件批量转换为512x512的PNG图像

    参数：
    - input_dir: 输入目录路径（包含.fit文件）
    - output_dir: 输出目录路径（保存.png文件）
    - target_size: 目标图像尺寸，默认512x512
    """
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 获取待处理文件列表
    file_list = [f for f in os.listdir(input_dir) if f.endswith('.fit')]

    # 使用tqdm创建进度条[6,8](@ref)
    with tqdm(total=len(file_list), desc="转换进度", unit="文件") as pbar:
        for filename in file_list:
            try:
                # 读取FITS文件[9,10](@ref)
                with fits.open(os.path.join(input_dir, filename)) as hdul:
                    data = hdul[0].data.astype(np.float32)

                    # 数据维度验证与处理
                    if len(data.shape) == 3:  # 处理三维数据（如RGB）
                        data = data.mean(axis=0)  # 取平均或选择特定通道
                    elif len(data.shape) != 2:
                        raise ValueError(f"非常规数据维度：{data.shape}")

                    # 数据归一化处理
                    data_normalized = (data - np.min(data)) / (np.max(data) - np.min(data)) * 255

                    # 创建图像对象并调整尺寸[1](@ref)
                    img = Image.fromarray(data_normalized.astype(np.uint8))
                    img_resized = img.resize(target_size, Image.LANCZOS)

                    # 保存PNG文件
                    output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.png")
                    img_resized.save(output_path)

                    # 更新进度描述[7](@ref)
                    pbar.set_postfix({"状态": "成功", "当前文件": filename[:15]+"..."})

            except Exception as e:
                pbar.set_postfix({"状态": f"失败: {str(e)}", "当前文件": filename[:15]+"..."})
            finally:
                pbar.update(1)

# 示例调用
if __name__ == "__main__":
    input_dir = "data_ori_fit"   # FITS文件目录
    output_dir = "data_moxing_png"  # 输出目录

    fits_to_png_512(input_dir, output_dir)

转换进度: 100%|██████████| 2500/2500 [01:23<00:00, 30.01文件/s, 状态=成功, 当前文件=galaxy_image_99...]


### 带宽涂污（ori.fit->dirty.fit）

In [23]:
# 相对路径：输入、输出目录
input_dir  = 'data_ori_fit'
output_dir = 'data_dirty_fit'
os.makedirs(output_dir, exist_ok=True)

# 模拟参数设置
frac_bandwidth = 0.2   # 宽带相对带宽 (例如20%)
num_channels = 5       # 模拟的频率通道数（在带宽内采样的频率点）
# UV覆盖相关参数
max_baseline_fraction = 0.5  # 最大基线长度相对于UV平面最大频率的比例 (决定分辨率, 0.5表示UV半径50%)
num_baselines = 2000         # 随机选择的基线数量（采样点数量，越少则UV覆盖越稀疏）

# 遍历输入目录下所有FITS图像文件
for file_path in glob.glob(os.path.join(input_dir, "*.fit*")):
    # 读取FITS图像数据
    hdul = fits.open(file_path)
    data = hdul[0].data.astype(np.float64)  # 使用float64提高FFT精度
    header = hdul[0].header
    hdul.close()
    # 如果图像有多于2维（例如包含多通道），压缩成2D图像
    if data.ndim > 2:
        data = data.squeeze()
    Ny, Nx = data.shape  # 图像的高度和宽度

    # 1. 对原始模型图像进行二维傅里叶变换，获取"真值"天空的频谱（能见度）数据
    #   - 先将图像进行 fftshift 使得图像中心对应相位中心，然后执行 FFT，再 fftshift 得到零频率在中心的频谱
    ft_image = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(data)))
    # 此时 ft_image 为复数数组，代表 UV 平面的理想完全采样情况

    # 2. 生成 UV 采样掩膜，模拟稀疏阵列的不完整 UV 覆盖
    uv_mask = np.zeros_like(ft_image, dtype=bool)
    u0 = Nx // 2  # UV平面中心 u 坐标
    v0 = Ny // 2  # UV平面中心 v 坐标
    max_r = min(u0, v0) * max_baseline_fraction  # 最大采样半径
    np.random.seed(42)  # 固定随机种子，确保每次运行采样一致（可根据需要移除以获得不同随机采样）

    # 随机生成基线点在 UV 平面的坐标偏移 (du, dv)，在圆形区域内均匀分布
    points = []
    count = 0
    while count < num_baselines:
        u_rand = (np.random.rand() * 2 - 1) * max_r
        v_rand = (np.random.rand() * 2 - 1) * max_r
        if u_rand**2 + v_rand**2 <= max_r**2:
            u_idx = int(np.round(u_rand))
            v_idx = int(np.round(v_rand))
            if abs(u_idx) <= u0 and abs(v_idx) <= v0:
                points.append((u_idx, v_idx))
                count += 1
    # 去除重复点和零点
    unique_points = set(points)
    if (0,0) in unique_points:
        unique_points.remove((0,0))
    # 对于每个选取的基线点，标记其正负对称点（保证共轭对称，以产生实值图像）
    for (du, dv) in list(unique_points):
        u_pos = u0 + du;  v_pos = v0 + dv
        u_neg = u0 - du;  v_neg = v0 - dv
        uv_mask[int(v_pos), int(u_pos)] = True
        uv_mask[int(v_neg), int(u_neg)] = True

    # 3. 模拟宽带频率涂污效应（bandwidth smearing）
    #    对于每条基线，在多个频率采样下获取能见度并取平均，模拟带宽内信号的相干平均
    observed_vis = np.zeros_like(ft_image, dtype=np.complex128)  # 存储观测到的(平均后)能见度
    f0 = 1.0  # 参考频率（归一化）
    freqs = f0 * (1 + np.linspace(-frac_bandwidth/2, frac_bandwidth/2, num_channels))  # 频率通道列表

    processed = set()
    for (du, dv) in unique_points:
        # 确保每对共轭基线只处理一次
        if (du, dv) in processed or (-du, -dv) in processed:
            continue
        processed.add((du, dv)); processed.add((-du, -dv))
        # 基线在 UV 平面上的基准坐标（以中心为0点偏移 du,dv）
        u_base = u0 + du;    v_base = v0 + dv
        u_base_neg = u0 - du; v_base_neg = v0 - dv
        if not (0 <= u_base < Nx and 0 <= v_base < Ny):
            continue
        # 收集该基线在各个频率的能见度值
        vis_vals = []
        vis_vals_neg = []
        for f in freqs:
            scale = f / f0  # 频率相对于 f0 的比例
            # 计算该频率下对应的 UV 坐标 (相同物理基线按频率缩放 UV距离)
            u_scaled = u0 + du * scale
            v_scaled = v0 + dv * scale
            u_scaled_neg = u0 - du * scale
            v_scaled_neg = v0 - dv * scale
            # 取与该坐标最接近的网格点的能见度值 (近似插值)
            u_idx = int(np.round(u_scaled)); v_idx = int(np.round(v_scaled))
            u_idx_neg = int(np.round(u_scaled_neg)); v_idx_neg = int(np.round(v_scaled_neg))
            if 0 <= u_idx < Nx and 0 <= v_idx < Ny:
                vis_vals.append(ft_image[v_idx, u_idx])
            else:
                vis_vals.append(0+0j)
            if 0 <= u_idx_neg < Nx and 0 <= v_idx_neg < Ny:
                vis_vals_neg.append(ft_image[v_idx_neg, u_idx_neg])
            else:
                vis_vals_neg.append(0+0j)
        # 频率平均能见度（复数平均）
        vis_avg = sum(vis_vals) / len(vis_vals) if len(vis_vals) > 0 else 0+0j
        vis_avg_neg = sum(vis_vals_neg) / len(vis_vals_neg) if len(vis_vals_neg) > 0 else 0+0j
        # 将平均后的能见度赋值回观测能见度矩阵
        observed_vis[int(v_base), int(u_base)] = vis_avg
        observed_vis[int(v_base_neg), int(u_base_neg)] = vis_avg_neg

    # 4. 逆傅里叶变换得到脏图
    #    - 未采样的位置在 observed_vis 中为0，导致合成波束(PSF)产生旁瓣
    dirty_image_complex = np.fft.ifft2(np.fft.ifftshift(observed_vis))
    dirty_image = np.real(np.fft.fftshift(dirty_image_complex))  # 取实部作为脏图强度分布

    # 5. 将生成的脏图写入FITS文件保存
    out_name = os.path.basename(file_path)
    out_path = os.path.join(output_dir, out_name)
    # 复制原始头信息，并添加历史记录说明
    header_dirty = header.copy()
    header_dirty.add_history("Simulated dirty map (sparse uv sampling + bandwidth smearing)")
    fits.writeto(out_path, dirty_image.astype(np.float32), header_dirty, overwrite=True)

    # （可选）可使用 matplotlib 在此处显示或保存图像以检查效果

print("Dirty map simulation completed.")


Dirty map simulation completed.


### dirty.fit->dirty.png

In [26]:
def fits_to_png_512(input_dir, output_dir, target_size=(512, 512)):
    """
    将指定目录下的FITS文件批量转换为512x512的PNG图像

    参数：
    - input_dir: 输入目录路径（包含.fit文件）
    - output_dir: 输出目录路径（保存.png文件）
    - target_size: 目标图像尺寸，默认512x512
    """
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 获取待处理文件列表
    file_list = [f for f in os.listdir(input_dir) if f.endswith('.fit')]

    # 使用tqdm创建进度条[6,8](@ref)
    with tqdm(total=len(file_list), desc="转换进度", unit="文件") as pbar:
        for filename in file_list:
            try:
                # 读取FITS文件[9,10](@ref)
                with fits.open(os.path.join(input_dir, filename)) as hdul:
                    data = hdul[0].data.astype(np.float32)

                    # 数据维度验证与处理
                    if len(data.shape) == 3:  # 处理三维数据（如RGB）
                        data = data.mean(axis=0)  # 取平均或选择特定通道
                    elif len(data.shape) != 2:
                        raise ValueError(f"非常规数据维度：{data.shape}")

                    # 数据归一化处理
                    data_normalized = (data - np.min(data)) / (np.max(data) - np.min(data)) * 255

                    # 创建图像对象并调整尺寸[1](@ref)
                    img = Image.fromarray(data_normalized.astype(np.uint8))
                    img_resized = img.resize(target_size, Image.LANCZOS)

                    # 保存PNG文件
                    output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_dirty.png")
                    img_resized.save(output_path)

                    # 更新进度描述[7](@ref)
                    pbar.set_postfix({"状态": "成功", "当前文件": filename[:15]+"..."})

            except Exception as e:
                pbar.set_postfix({"状态": f"失败: {str(e)}", "当前文件": filename[:15]+"..."})
            finally:
                pbar.update(1)

# 示例调用
if __name__ == "__main__":
    input_dir = "data_dirty_fit"   # FITS文件目录
    output_dir = "data_dirty_png"  # 输出目录

    fits_to_png_512(input_dir, output_dir)

转换进度: 100%|██████████| 2500/2500 [01:51<00:00, 22.43文件/s, 状态=成功, 当前文件=galaxy_image_99...]
